In [ ]:
from pathlib import Path
import csv
import json
import math
import statistics
from typing import List, Dict

import matplotlib.pyplot as plt

# Configure: point this to the folder holding CSV files for ONE run
DATA_DIR = 'run-001/parameter-set-1/single-run'

def read_csv_folder(folder: str) -> List[Dict]:
    rows = []
    print(f"Reading CSV files from folder: {folder}")
    for p in sorted(Path(folder).glob("*.csv")):
        with p.open("r", newline="") as f:
            reader = csv.DictReader(f)
            for row in reader:
                # Convert types where sensible
                row["ok"] = row["ok"] in ("1", "True", "true")
                for k in ("e2e_ms", "server_latency_ms", "timeout_ms"):
                    if row.get(k):
                        try:
                            row[k] = float(row[k])
                        except Exception:
                            row[k] = None
                for k in ("delay_ms", "jitter_ms", "payload_kb", "concurrency", "requests"):
                    if row.get(k):
                        try:
                            row[k] = int(float(row[k]))
                        except Exception:
                            pass
                rows.append(row)
    return rows

results = read_csv_folder(DATA_DIR)
len(results)

In [ ]:
# Summary metrics for the single run
def summarize(results: List[Dict]) -> Dict[str, float]:
    n = len(results)
    ok = sum(1 for r in results if r.get("ok"))
    e2e_vals = [r["e2e_ms"] for r in results if isinstance(r.get("e2e_ms"), (int, float))]
    success_rate = ok / n if n else float("nan")
    mean_e2e = statistics.mean(e2e_vals) if e2e_vals else float("nan")
    p95 = (sorted(e2e_vals)[max(0, int(0.95 * len(e2e_vals)) - 1)] if e2e_vals else float("nan"))
    return {"n": n, "ok": ok, "success_rate": success_rate, "mean_e2e_ms": mean_e2e, "p95_e2e_ms": p95}

summary = summarize(results)
summary

In [ ]:
# Histogram of end-to-end latency
e2e_vals = [r["e2e_ms"] for r in results if isinstance(r.get("e2e_ms"), (int, float))]
plt.figure()
plt.hist(e2e_vals, bins=30)
plt.xlabel("End-to-end latency (ms)")
plt.ylabel("Count")
plt.title("Single Run: E2E Latency Distribution")
plt.show()

In [ ]:
# Request-by-request latency (trace)
ids = [int(r["req_id"]) if r.get("req_id") not in (None, "") else i for i, r in enumerate(results)]
e2e_vals = [r["e2e_ms"] for r in results if isinstance(r.get("e2e_ms"), (int, float))]

plt.figure()
plt.plot(ids[:len(e2e_vals)], e2e_vals, marker=".")
plt.xlabel("Request index")
plt.ylabel("E2E latency (ms)")
plt.title("Single Run: Request Latency Trace")
plt.show()